### Generate an RSA keypair

We start this lab by generating rsa keypairs. There are many tools available to help you generate key pairs. In this lab we will start with ssh-keygen. It is available on Linux, Mac and Windows.

------------


### Creating a public-private key pair

Run the openssl command to create a private RSA key. Then run the second command to extract the public key

In [1]:
!openssl genrsa -out key.pem 4096

In [2]:
!openssl rsa -in key.pem -pubout > key.pub

writing RSA key


### Using public-private key pair

The two basic uses of public-key cryptography are to encrypt something or to sign something. But we typically never use asymmetric encryption to encrypt files.

Why? Give 2 reasons.

Run the openssl command to encrypt a file. Use your password file as the data to encrypt.

In [3]:
!openssl pkeyutl -in /etc/passwd -out ./passwd.ctxt -inkey ./key.pub -pubin -encrypt

Public Key operation error
20F04A97FFFF0000:error:0200006E:rsa routines:ossl_rsa_padding_add_PKCS1_type_2_ex:data too large for key size:../crypto/rsa/rsa_pk1.c:129:


This generated a "Public Key operation error". Why was that?

Look at the size of the file. `!wc -m /etc/passwd`

In [17]:
!wc -m /etc/passwd

2993 /etc/passwd


Does that help?

To find the size of the file in bytes I could run the cell below.

In [18]:
!ls -l /etc/passwd

-rw-r--r-- 1 root root 2993 Mar  8 20:39 /etc/passwd


This is weird though because the character and byte size are the same. 

If the file is encoded using ASCII, which means that the file size in bytes and character count will be the same

In [19]:
!file /etc/passwd

/etc/passwd: ASCII text


Password files typically only contain ASCII characters.
ASCII is a character encoding standard that assigns each character a unique numberical code.

Typcially we use public-key encryption to encrypt something small, like the digest (hash value) of the file.

Use the openssl command to generate the sha2-256 digest for your password file (/etc/passwd)<br>
`!openssl dgst -sha2-256` <br>


In [9]:
!cat /etc/passwd | openssl dgst -sha2-256 -out ./passwd.dgst

Now take a look at the digest.

In [5]:
%cat ./passwd.dgst

SHA2-256(stdin)= c4b24f307830cb1b8ea69a5bd74f0c9d52f118ea2c5d4de06335d59ba70e6f4e


In [6]:
!openssl pkeyutl -in ./passwd.dgst -out ./passwd.cdgst -inkey ./key.pub -pubin -encrypt

Now take a look at the output.

In [7]:
!cat ./passwd.cdgst

���X���vSnO�Ⱥel�b9���Dq�[[�����Ud�����$��kb���w����Z�2�ԝ��4��F��
�����I�a�5D���*�^�6}"��O�HBYi�ޞv���� ���:���QiMK�{y���ՠ��q]Vx�]�� ���͜t�����S��������'����9Hagw=��O�=�gޕ�◫��TڠEx��hw��K[�2����<�$)D���{`��÷�#��<@;�?V����R�b3|whV2yD�`ɠ����N1�c2��v��n׏�����Z�U��j�<n�;h��e��ǮK��)��D�]�+jA�)\z�C�4|]���
G�
}�A� �����+�D� ^=�h��+~���Ror@H��-���%.J<L�f���2�X����� ��6e]���7f��S_�r���=��Z �B�\!}S�^�q�?��7��.m�('s��B����6�B�FЌDޅ��}

Rerun the command to encrypt the digest and then review the output. What do you notice? Explain.

In [9]:
!openssl pkeyutl -in ./passwd.dgst -out ./passwd.cdgst -inkey ./key.pub -pubin -encrypt

In [10]:
!cat ./passwd.cdgst

P ��X6������bO�M�ʌAe��z�E����;C7���B��;z�T3��g4�
��I9(2�;��_��0��n8�_��������w�L*%p�yI���Q�E�U�l(�}^A������ ��t@ �3�F1�2���%KZ��l�x� ��w����N��UPr
��SO��Y��/y0`���,�S�T�bY+q;:ۃpԍ�uW�2�0���hֈz%v��:f	F Ԅx����:�+��y+��ݭ�8t���2�D��;�
&�b�u�װ4�fv�y"
�K����F���ş�۝k7�V.�?U�' �.�,>ވ�	����֭'�z��ϗ����;��{k�SV�

The output is NOT identical, it is different.
The output is shorter.

The encryption process uses a randomized padding scheme to make it difficult for an attacker to determine the original message from the encrypted output. This padding scheme uses a random value each time the encryption is performed. As a result, even if the same message is encrypted multiple times with the same key, the resulting output will be different each time.

Therefore, the fact that the output is different when the command is run again is expected behavior due to the randomization involved in the encryption process.

---
## Digital Signatures

Asymmetric cryptography can also be used to digitally sign data, but here you use the private key, and just like encrypting, the size of the data you can sign is limited.

In [14]:
!openssl dgst -sha2-256 -sign key.pem -out passwd.sign /etc/passwd

In [15]:
!cat ./passwd.sign

��[.����Sج���T���9��%�R���Y
�'hd�
�F�aSKc�cn��o7�'`���B��^�(>W�7�u 4�ɮjm*�d�q{�9���7� ~�����5�d�wĢ��h��\�K�o�ͼ�$���Ђ�a�Y�M�1,�:��cJ�
�ݘk�!�����,���S��j3�1��uOkʜ7R+��c|�1x���7�T��Q�t*��<B�}�����B��1�6�.5��~<�%�
��O�/�fB
� �<g�w�N��xL�#P���h�u�yҨ��//�n�֩%�?k��' %�r�O@�"U������:8$4�D�����D]&��W�ۦ�@�EDe�vI����$���"
V7��j�'fKkO`ʘZ��\C�ȯ4d�8#u$���k���V�/��ʭӉٶ`�t��XM�2�:��#G%a	<W㇢Z��]D�����B�=�C>��婜U@h�I�{�

Now verify the file:<br>
`!openssl dgst -verify ./key.pub -keyform pem -sha256 -signature passwd.sign -binary /etc/passwd`


In [16]:
!openssl dgst -verify ./key.pub -keyform pem -sha256 -signature passwd.sign -binary /etc/passwd


Verified OK


----------
### This concludes the lab.

Use File > Save and Export Notebook as... > PDF